In [1]:
import re

import numpy as np
import pandas as pd

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [181]:
base_data = pd.read_pickle('./01_base_data.pkl')
base_data[0:2]

,Category,Scientific name,Helps,Helped by,Attracts,-Repels/+distracts,Avoid,Comments,combined_name
Common name,,,,,,,,,
alliums,vegetables,allium,"fruit trees,, nightshades ,tomatoes,, capsicum...","carrots,, tomatoes,, carrots,african spider pl...",thrips,"-rabbits,, slugs ,see garlic,, -aphids,, carro...","beans, peas,,",alliums are a family of plants which include o...,allium | alliums
asparagus,vegetables,asparagus officinalis,"tomatoes,, parsley,,","aster family flowers, dill, coriander, tomatoe...",coupled with basil seems to encourage lady bugs,,"onion, garlic, potatoes, gladiolus",,asparagus officinalis | asparagus


In [182]:
am_len = len(base_data)
index = list(base_data.index)
index.sort()
am = pd.DataFrame(np.zeros(shape=(am_len, am_len)),
                  columns=index,
                  index=index)

In [183]:
for index, row in base_data[0:15].iterrows():
    helps = [x.rstrip().lstrip() for x in row['Helps'].split(',') if x != '']
    helped_by = [x.rstrip().lstrip() for x in row['Helped by'].split(',') if x != '']
    hurt_by = [x.rstrip().lstrip() for x in row['Avoid'].split(',') if x != '']
    
    attracts = [x.rstrip().lstrip() for x in row['Attracts'].split(',') if x != '']

    repels_and_distracts = [x.rstrip().lstrip() for x in row['-Repels/+distracts'].split(',') if x != '']
    repels = [x for x in repels_and_distracts if x[0:1] == '-']
    distracts = [x for x in repels_and_distracts if x[0:1] != '-']
    
    
    for x in helps:
        if x in am.index:
            am.at[index, x] += 1
    
    for x in helped_by:
        if x in am.index:
            am.at[x, index] += 1
    
    for x in hurt_by:
        if x in am.index:
            am.at[x, index]  -= 1
    

In [184]:
am

,alfalfa,alliums,alyssum,anise,apple,apricot,asparagus,baby's breath,basil,"beans, bush","beans, fava","beans, pole",bee balm,beets,blueberries,borage,brassicas,broccoli,brussels sprouts,cabbage,californian poppy,caraway,carrots,catnip,cauliflower,celery,chamomile,chard,chervil,chives,cilantro / coriander,corn / maize,cucumber,cucurbits,dianthus,dill,eggplant or aubergine,fennel,flax,fruit trees,garlic,geraniums,grapes,hyssop,kohlrabi,larkspur,lavender,leek,legumes,lemon grass,lettuce,lovage,lupin,marigold,melon,mustard,nasturtium,nightshades,okra,onion,oregano,pansy,parsley,parsnip,passion fruit,peanut,pears,peas,peppermint,peppers,petunia,phacelia,potato,pumpkin,radish,rose,rosemary,rutabegas,sage,southernwood,soybean,spearmint,spinach,squash,stinging nettle,strawberries,summer savoury,sunflower,swan plant,sweet pea,sweet potato,tansy,tarragon,thyme,tomatoes,turnips,walnut tree,wormwood,yarrow,zinnia
alfalfa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alliums,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
alyssum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
anise,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
apple,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
apricot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
asparagus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
baby's breath,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [185]:
def affects_growth(common_plant_name):
    return am[common_plant_name][am[common_plant_name]!=0.0]

def growth_affected_by(common_plant_name):
    return am.loc[common_plant_name][am.loc[common_plant_name] != 0.0]

In [186]:
affects_growth('alliums').to_dict()

{'carrots': 3.0, 'peas': -1.0, 'tomatoes': 1.0}

In [187]:
growth_affected_by('turnips').to_dict()

{}

In [190]:
am.to_pickle('./02_adjacency_matrix.pkl')